# PhaseObjC - Stoichiometric Phase Example 1
## Illustrate Objective-C type library interaction and plotting
Required Python code to load the phase library. 

In [ ]:
from thermoengine import phases
from thermoengine import model
from thermoengine import chem

import numpy as np
from scipy import optimize

In [ ]:
modelDB = model.Database()
modelDB.phase_attributes.keys()

In [ ]:
relevant_phases = ['Cpx', 'Grt', 'Ol', 'Opx', 'Bt', 'Crn', 'Rt', 'Fsp', 'Cam',
                   'Zo', 'Crd', 'Ms', 'Qz', 'Ky', 'Sil', 'SplS', 'Mll', 'Nph', 'Ilm']

oxide_num = chem.oxide_props['oxide_num']
all_mol_oxide_comp = np.zeros((0,oxide_num))
all_phase_name = np.zeros((0))
all_endmember_name = np.zeros((0))
all_endmember_ind = np.zeros((0))
all_phase_ind = np.zeros((0))
                             
for (ind_phs, iabbrev) in enumerate(relevant_phases):
    iphs_props = modelDB.phase_attributes[iabbrev]['props']
    #print(iphs_props)
    iall_mol_oxide_comp = iphs_props['mol_oxide_comp']
    iendmember_name = iphs_props['endmember_name']
    iendmember_num = len(iendmember_name)
    
    iendmember_ind = np.arange(iendmember_num)
    
    iphase_name_tile = np.tile(np.array([iphs_props['phase_name']]), iendmember_num)

    all_phase_ind = np.hstack((all_phase_ind, np.tile(ind_phs,(iendmember_num))))
    all_mol_oxide_comp= np.vstack((all_mol_oxide_comp, iall_mol_oxide_comp))
    all_phase_name = np.hstack((all_phase_name, iphase_name_tile))
    all_endmember_name = np.hstack((all_endmember_name, iendmember_name))
    all_endmember_ind = np.hstack((all_endmember_ind, iendmember_ind))


In [ ]:

import matplotlib.pyplot as plt
%matplotlib notebook


plt.imshow(all_mol_oxide_comp.T,cmap='viridis')
plt.clim(0,1)
plt.colorbar()


In [ ]:
oxide_atom_num = chem.oxide_props['cat_num']+chem.oxide_props['oxy_num']

In [ ]:
oxide_comp_per_atom = all_mol_oxide_comp/np.tile(oxide_atom_num[np.newaxis,:],(50,1))

In [ ]:
oxide_comp_per_atom /= np.tile(np.sum(oxide_comp_per_atom, axis=1)[:,np.newaxis],(1,oxide_atom_num.size))

In [ ]:
np.sum(oxide_comp_per_atom,axis=1)

In [ ]:
plt.figure()
plt.imshow(oxide_comp_per_atom.T,cmap='viridis')
plt.clim(0,1)
plt.colorbar()

In [ ]:
Nendmember, Noxide = oxide_comp_per_atom.shape

In [ ]:
u, s, vh =np.linalg.svd(oxide_comp_per_atom.T)

In [ ]:
TOL = 1e-4
N_nonrxn = np.sum(np.abs(s)>= TOL)
N_rxn = Nendmember-Noxide + np.sum(np.abs(s)< TOL)
non_rxn = vh[0:N_nonrxn]
rxn_svd = vh[N_nonrxn:]

scl = np.array([np.sum(np.abs(irxn)) for irxn in rxn])
rxn_svd = rxn_svd/np.tile(scl[:,np.newaxis],(1,50))

In [ ]:
plt.figure()
plt.imshow(rxn_svd,cmap='bwr')
plt.clim(-.5,.5)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

In [ ]:
def rxn_entropy(irxn, TOL=1e-10):
    # scl = irxn[irxn>0].sum()
    # irxn /= scl
    irxn_abs = np.abs(irxn)
    xlogx = irxn_abs*np.log(irxn_abs)
    xlogx[irxn_abs<TOL] = 0
    entropy = -np.sum(xlogx)
    return entropy

In [ ]:
rxn_entropy(rxn[0])

In [ ]:
cost_svd = np.array([rxn_entropy(irxn) for irxn in rxn_svd])
cost_svd

In [ ]:
def linear_combo(wts, rxn=rxn, return_scl=False):
    wts = np.array(wts)
    # print(wts.shape)
    scl = np.tile(wts[:,np.newaxis],(1,50))
    rxn_wt = np.sum(scl*rxn, axis=0)
    # scl_tot = np.sum(np.abs(rxn_wt))
    scl_tot = np.linalg.norm(rxn_wt)
    rxn_wt /= scl_tot
    
    if return_scl:
        return rxn_wt, wts/scl_tot
    
    else:
        return rxn_wt



def combine_rxn(wts, rxn=rxn):
    rxn_wt = linear_combo(wts, rxn=rxn)
    # scl_tot = rxn_wt[rxn_wt>0].sum()
    
    # scl_tot = 0.5*np.sum(np.abs(rxn_wt))
    # rxn_wt /= scl_tot
    # print(rxn_wt)
    entropy = rxn_entropy(rxn_wt)
    # cost = entropy + 1000*(scl_tot-1)**2
    cost = entropy
    return cost

In [ ]:
np.max

In [ ]:
def ortho_penalty(wts, wtcoefs_ortho=None, scale=1):
    if wtcoefs_ortho is None:
        cost = 0
    else:
        wts_ortho = get_wtcoefs_ortho(wts, wtcoefs_ortho, 
                                      apply_norm=False)
        frac = np.minimum(np.linalg.norm(wts_ortho)/np.linalg.norm(wts), 1)
        
        cost = scale*(1-frac**2)
        # cost = scale*np.sqrt(1 - frac**2)
        # print('ortho_penalty = ', cost)
    
    return cost
        
def combine_rxn_costfun(wts, rxn=rxn, wtcoefs_ortho=None, scale=1, debug=False):
    cost = combine_rxn(wts, rxn=rxn) + ortho_penalty(
        wts, wtcoefs_ortho=wtcoefs_ortho, scale=scale)
    
    if debug:
        if wtcoefs_ortho is not None:
            wts_ortho = get_wtcoefs_ortho(wts, wtcoefs_ortho, apply_norm=False)
            print('norm(wts) = {wts_norm}'.format(wts_norm=np.linalg.norm(wts)))
            print('norm(ortho) = {ortho_norm}'.format(ortho_norm=np.linalg.norm(wts_ortho)))
            ortho_cost = ortho_penalty(wts, wtcoefs_ortho=wtcoefs_ortho, scale=scale)
            print('ortho_cost = ', ortho_cost)
        
    return cost

In [ ]:
rxn_svd.shape

In [ ]:
np.seterr(all='ignore')

In [ ]:


def random_rxn(wtcoefs_ortho=None, Nbasis=36):
    wts = 2*np.random.rand(Nbasis)-1
    wts /= np.linalg.norm(wts)
    
    if wtcoefs_ortho is not None:
        wts = get_wtcoefs_ortho(wts, wtcoefs_ortho)

    
    return wts
    
def draw_basic_rxns(rxn_svd, wtcoefs_ortho=None, Ndraw=10, ortho_scale=1):
    Nbasis = rxn_svd.shape[0]
    Nendmem = rxn_svd.shape[1]
    
    wtcoefs = np.zeros((Ndraw, Nbasis))
    cost = np.zeros(Ndraw)
    rxn_coefs = np.zeros((Ndraw, Nendmem))
    
    
    
    for ind in range(Ndraw):
        iwts0 = random_rxn(wtcoefs_ortho=wtcoefs_ortho, Nbasis=Nbasis)
        
        def costfun(wts, rxn=rxn_svd, wtcoefs_ortho=wtcoefs_ortho,
                scale=ortho_scale):
            return combine_rxn_costfun(wts, rxn=rxn, 
                                       wtcoefs_ortho=wtcoefs_ortho, 
                                       scale=ortho_scale)

        for ind_fit in range(1):
            
            ifit = optimize.minimize(costfun, iwts0, tol=1e-10)
            iwts0 = ifit['x']
        
        
        iwt_fit = ifit['x']
        combine_rxn_costfun(iwt_fit, rxn=rxn_svd, wtcoefs_ortho=wtcoefs_ortho, scale=ortho_scale, debug=True)
        # ifit = optimize.minimize(combine_rxn, ifit['x'], tol=1e-10) 
        
        irxn_coefs = linear_combo(iwt_fit, rxn=rxn_svd)
        wtcoefs[ind] = iwt_fit
        rxn_coefs[ind] = irxn_coefs
        cost[ind] = ifit['fun']
    
    return wtcoefs, rxn_coefs, cost

def next_basic_rxn(rxn_svd, wtcoefs_ortho=None, Ndraw=10, ortho_scale=1):
    wtcoefs, rxn_coefs, cost = draw_basic_rxns(
        rxn_svd, wtcoefs_ortho=wtcoefs_ortho, Ndraw=Ndraw, 
        ortho_scale=ortho_scale)
    ind = np.argmin(cost)
    
    return wtcoefs[ind], rxn_coefs[ind], cost[ind]

def get_wtcoefs_ortho(wtcoefs, wtcoefs_ortho, apply_norm=True):  
    wts_ortho = wtcoefs.copy()
    for iwtcoefs_ortho in wtcoefs_ortho:
        wts_ortho -= np.dot(
                iwtcoefs_ortho, wts_ortho)*iwtcoefs_ortho
    
    if apply_norm:
        wts_ortho /= np.linalg.norm(wts_ortho)
        
    return wts_ortho


def get_basic_rxns(rxn_svd, Ndraw=2, ortho_scale=1):
    Nbasis = rxn_svd.shape[0]
    Nendmem = rxn_svd.shape[1]
    
    wtcoefs_ortho = np.zeros((Nbasis, Nbasis))
    wtcoefs = np.zeros((Nbasis, Nbasis))
    rxn_coefs = np.zeros((Nbasis, Nendmem))
    costs = np.zeros(Nbasis)
    
    for ind in range(Nbasis):
        iwtcoefs, irxn_coefs, icost = next_basic_rxn(
            rxn_svd, wtcoefs_ortho=wtcoefs_ortho, Ndraw=Ndraw,
            ortho_scale=ortho_scale)
        
        iwtcoefs_ortho = get_wtcoefs_ortho(iwtcoefs, wtcoefs_ortho)
        
        wtcoefs[ind] = iwtcoefs
        wtcoefs_ortho[ind] = iwtcoefs_ortho
        rxn_coefs[ind] = irxn_coefs
        costs[ind] = icost
        # print(icost, np.round(irxn_coefs, decimals=1))
        print('icost({ind}) = {icost}'.format(ind=ind,icost=icost))
        print('=====')
        
    return wtcoefs, costs, rxn_coefs, wtcoefs_ortho

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = get_basic_rxns(rxn_svd, Ndraw=10, ortho_scale=10)

In [ ]:
info_cost = [combine_rxn(iwts, rxn=rxn_svd) for iwts in wtcoefs]

plt.figure()
plt.plot(cost_svd, 'ko')
plt.plot(costs, 'rx')
plt.plot(info_cost,'ro')

plt.figure()
plt.plot(rxn_coefs[::10].T, '-')

In [ ]:
plt.figure()
plt.imshow(rxn_coefs,cmap='bwr')
plt.clim(-.5,.5)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

plt.figure()
plt.imshow(rxn_svd,cmap='bwr')
plt.clim(-.5,.5)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

In [ ]:
plt.figure()
plt.imshow(np.dot(rxn_coefs, rxn_coefs.T))
plt.colorbar()

In [ ]:
np.linalg.norm(rxn_coefs[-1])

In [ ]:
mol_oxide_comp = Grt.endmember_props['mol_oxide_comp']
endmember_names = Grt.endmember_props['names']
for iendmember,imol_oxide_comp in zip(endmember_names, mol_oxide_comp):
    print('========')
    print(iendmember)
    print('----')
    for imol_oxide, ioxide in zip(imol_oxide_comp, chem.oxide_props['oxides']):
        print(ioxide, ' = ', imol_oxide)

In [ ]:
from thermoengine import chem
chem.oxide_props

In [ ]:
chem_pot[0]

In [ ]:
modelDB.phase_attributes['Aeg']

In [ ]:
modelDB.phase_attributes['Grt']

In [ ]:
info=modelDB.phase_details['info']['pure']
abbrev = 'Aeng'
this_info = info.loc[info['Abbrev']==abbrev]

In [ ]:
# phases.get_phaselist()

In [ ]:
modelDB.phase_details['info']['solution'].head()

In [ ]:
modelDB.phase_details['info']['pure'].head()

In [ ]:
modelDB.phase_details['info']['active_pure'].head()

In [ ]:
modelDB._phase_cls

In [ ]:
garnet = phases.SolutionPhase('GarnetBerman','Grt')
# garnet_HP = phases.SolutionPhase('HollandAndPowell','Grt')
garnet_Stix = phases.SolutionPhase('GarnetStixrude','Grt')

In [ ]:
display(garnet.props)
display(garnet_Stix.props)

In [ ]:
display(garnet.endmember_props)
display(garnet_Stix.endmember_props)